**MAESTRÍA EN INTELIGENCIA ARTIFICIAL APLICADA**

**Curso: Operaciones de Aprendizaje Automático**

Tecnológico de Monterrey

**Fase 1 – Procesamiento y Modelado Inicial**
En esta primera etapa aprenderás a trabajar con los datos y a construir tus primeros modelos:

* Realizar un análisis del problema y generar una propuesta de valor.
* Manipular y preparar datos.
* Explorar y preprocesar información.
* Aplicar técnicas de versionado para asegurar reproducibilidad y trazabilidad.
* Construir, ajustar y evaluar modelos de Machine Learning.

---

**EQUIPO 10**

**ALUMNOS:**

Gabriela Lizeth Moreno Hernández - AXXXXX

Alejandro Chávez Campos  -  AXXXXX

Carlos Armando Alcántar Sánchez - A01311747

Luis Fernando Caporal Montes de Oca  -  A01795898

Yamil Abraham Nechar Reyes - A01795726

# Caravan Insurance

Dataset proporcionado por la compañía holandesa de minería de datos Sentient Machine Research. El objetivo principal es predecir qué clientes tienen una póliza de seguro para caravanas.

### Características principales:

* Fuente: Datos de clientes reales, anonimizados y preparados para una competición de minería de datos.

* Volumen: Contiene información de más de 9.000 clientes en total.

* Variables: Cada cliente se describe mediante 86 atributos.

Tipos de Datos:

* **Datos Sociodemográficos (Atributos 1-43)**: Información derivada de los códigos postales de los clientes. Todos los residentes de una misma zona postal comparten estos mismos atributos (ej. porcentaje de hogares alquilados, distribución por nivel de ingresos, etc.).

* **Datos de Uso de Productos** (Atributos 44-86): Reflejan la posesión o uso de otros productos y servicios por parte del cliente.

* **Variable Objetivo:** El atributo 86, "CARAVAN: Number of mobile home policies", indica si un cliente tiene o no un seguro de caravana.

### Estructura de los Archivos:

El proyecto se divide en tres archivos con campos separados por tabuladores:

##### TICDATA2000.txt (Conjunto de Entrenamiento):

Registros: 5.822 clientes.

Contenido: Incluye los 86 atributos completos, incluida la variable objetivo. Se utiliza para desarrollar, entrenar y validar los modelos predictivos.

##### TICEVAL2000.txt (Conjunto de Evaluación):

Registros: 4.000 clientes.

Contenido: Tiene el mismo formato que el archivo de entrenamiento, pero no incluye la variable objetivo. Este archivo se utiliza para realizar las predicciones finales con el modelo ya construido.

#####  TICTGTS2000.txt (Soluciones del Conjunto de Evaluación):

Contenido: Contiene las respuestas reales (targets) para los 4.000 clientes del conjunto de evaluación. Solo los organizadores tenían esta información, permitiendo así la evaluación objetiva de las predicciones realizadas por los participantes.

Nota Importante: Todas las variables cuyo nombre comienza por "M" son variables de código postal que proporcionan información sobre la distribución de una característica específica en el área de residencia del cliente.

In [1]:
# --- Librerías ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import pointbiserialr
from sklearn.preprocessing import PowerTransformer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from pycaret.classification import predict_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from pycaret.classification import setup, compare_models
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.base import BaseEstimator
import os

# Configuración de gráficos
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)

In [1]:
os.chdir("..")  # subir al nivel raíz del proyecto

!pwd  # verificar ruta actual
!dvc pull data/processed/insurance_company_processed_data.csv.dvc

os.chdir("notebooks")

/Users/luisfernandocaporalmontesdeoca/Documents/MNA-V Tec de Monterrey/MLOps/MNA_MLOps_Equipo12_backup
Fetching
!
  0% Checking cache in '/Users/luisfernandocaporalmontesdeoca/Documents/MNA-V Te
Fetching                                                                        
Building workspace index                              |3.00 [00:00,  854entry/s]
Comparing indexes                                    |4.00 [00:00, 3.03kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
Everything is up to date.


In [2]:
import logging
import pandas as pd
from scipy.stats import pointbiserialr
from sklearn.preprocessing import PowerTransformer

# Configuración del logger
logger = logging.getLogger('preprocessor_logger')
logger.setLevel(logging.INFO)

if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - [%(funcName)s] - %(message)s'
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)


class Preprocessor:
    def __init__(self, df: pd.DataFrame, target_col: str = 'target'):
        """
        Inicializa el preprocesador con el DataFrame original.
        
        Args:
            df: DataFrame a procesar
            target_col: Nombre de la columna target
        """
        self.target_col = target_col
        self.original_df = df.copy()
        self.processed_df = None
        self.cor_target = None
        self.product_cols = None
        
        # Atributos para exportar
        self.zones_ = None
        self.zone_mapper_df = None
        self.cols_to_drop_ = []
        self.power_params_ = None
        self.skewed_cols_ = []
        self.high_corr_df_products = None
        
        logger.info(f"Preprocessor inicializado - Shape original: {self.original_df.shape}")

    def _log_shape_change(self, previous_shape: tuple, operation: str):
        """Registra cambios en el shape del DataFrame."""
        current_shape = self.processed_df.shape
        logger.info(
            f"{operation} - Shape cambiado de {previous_shape} a {current_shape} "
            f"(Filas: {previous_shape[0]} → {current_shape[0]}, "
            f"Columnas: {previous_shape[1]} → {current_shape[1]})"
        )

    def group_sociodemographic_cols(self, sociodemographic_cols: list):
        """
        Agrupa columnas sociodemográficas en una sola columna 'zone'.
        
        Args:
            sociodemographic_cols: Lista de columnas sociodemográficas a agrupar
        """
        logger.info(f"Iniciando agrupación de columnas sociodemográficas: {sociodemographic_cols}")
        
        previous_shape = self.original_df.shape
        
        # Crear columna zone basada en agrupación
        self.original_df['zone'] = self.original_df.groupby(sociodemographic_cols).ngroup() + 1
        
        # Crear mapper de zonas
        expanded_cols = sociodemographic_cols + ['zone']
        self.zone_mapper_df = self.original_df[expanded_cols].drop_duplicates()
        
        self.zones_ = [str(col) for col in self.zone_mapper_df['zone'].unique()]
        
        # Eliminar columnas originales
        self.original_df.drop(columns=sociodemographic_cols, inplace=True)
        self.processed_df = self.original_df.copy()
        
        logger.info(f"Agrupación completada - {len(self.zone_mapper_df)} zonas únicas creadas")
        self._log_shape_change(previous_shape, "Agrupación sociodemográfica")

    def remove_duplicates(self):
        """Elimina duplicados exactos del DataFrame."""
        logger.info("Iniciando eliminación de duplicados exactos")
        
        previous_shape = self.processed_df.shape
        initial_rows = len(self.processed_df)
        
        self.processed_df = self.processed_df.drop_duplicates()
        final_rows = len(self.processed_df)
        removed_rows = initial_rows - final_rows
        
        logger.info(f"Duplicados eliminados: {removed_rows} filas removidas")
        self._log_shape_change(previous_shape, "Eliminación de duplicados exactos")

    def handle_complex_duplicates(self):
        """
        Maneja duplicados complejos donde las filas son idénticas excepto por el target.
        Conserva la moda del target en casos de múltiples duplicados.
        """
        logger.info("Iniciando manejo de duplicados complejos")
        
        df = self.processed_df.copy()
        previous_shape = df.shape
        initial_rows = len(df)
        
        # Agrupar filas duplicadas excepto por target
        columns = list(set(df.columns) - {self.target_col})
        grupos_duplicados = df.groupby(columns).groups
        
        indices_a_eliminar = []
        grupos_procesados = 0
        
        for fila, indices in grupos_duplicados.items():
            if len(indices) > 1:  # Solo grupos con duplicados
                grupos_procesados += 1
                grupo_actual = df.loc[indices]
                
                if len(indices) == 2:
                    # Eliminar todo el grupo (ambas filas)
                    indices_a_eliminar.extend(indices)
                else:
                    # Grupos con más de 2 filas: conservar solo la moda
                    moda_target = grupo_actual[self.target_col].mode()
                    
                    if len(moda_target) > 0:
                        moda = moda_target[0]
                        # Conservar solo las filas con target = moda
                        filas_a_eliminar = grupo_actual[grupo_actual[self.target_col] != moda].index
                        indices_a_eliminar.extend(filas_a_eliminar)
                    else:
                        # Si no hay moda clara, eliminar todo el grupo
                        indices_a_eliminar.extend(indices)
        
        # Crear nuevo DataFrame sin los duplicados problemáticos
        self.processed_df = df.drop(indices_a_eliminar)
        final_rows = len(self.processed_df)
        removed_rows = initial_rows - final_rows
        
        logger.info(
            f"Manejo de duplicados complejos completado - "
            f"{grupos_procesados} grupos procesados, {removed_rows} filas eliminadas"
        )
        self._log_shape_change(previous_shape, "Manejo de duplicados complejos")

    def get_correlations(self):
        """Calcula correlaciones punto-biserial entre features y target."""
        logger.info("Calculando correlaciones con el target")
        
        if self.target_col not in self.processed_df.columns:
            raise ValueError(f"Target column '{self.target_col}' no encontrada en el DataFrame")
        
        cor_target = {}
        features = [col for col in self.processed_df.columns if col != self.target_col]

        for col in features:
            corr, _ = pointbiserialr(self.processed_df[col], self.processed_df[self.target_col])
            cor_target[col] = corr

        self.cor_target = pd.Series(cor_target).sort_values(key=abs, ascending=False)
        
        logger.info(
            f"Correlaciones calculadas - "
            f"Rango: [{self.cor_target.min():.3f}, {self.cor_target.max():.3f}], "
            f"Top 3: {self.cor_target.head(3).to_dict()}"
        )

    def get_high_pair_correlations(self, product_cols: list):
        """
        Identifica pares de variables con alta correlación entre sí.
        
        Args:
            product_cols: Lista de columnas de productos a analizar
        """
        logger.info(f"Buscando correlaciones altas entre {len(product_cols)} columnas de productos")
        
        # Verificar que las columnas existen
        missing_cols = set(product_cols) - set(self.processed_df.columns)
        if missing_cols:
            raise ValueError(f"Columnas no encontradas: {missing_cols}")
        
        corr_abs_products = self.processed_df[product_cols].corr().abs()

        high_corr_pairs_products = []
        for i in range(len(corr_abs_products.columns)):
            for j in range(i+1, len(corr_abs_products.columns)):
                correlation = corr_abs_products.iloc[i, j]
                if correlation > 0.7:
                    high_corr_pairs_products.append({
                        'var1': corr_abs_products.columns[i],
                        'var2': corr_abs_products.columns[j], 
                        'correlation': correlation
                    })

        self.high_corr_df_products = pd.DataFrame(high_corr_pairs_products).sort_values(
            'correlation', ascending=False
        )
        
        # Filtrar correlaciones muy altas
        high_corr_count = len(self.high_corr_df_products[self.high_corr_df_products['correlation'] > 0.95])
        self.product_cols = product_cols
        
        logger.info(
            f"Análisis de correlación completado - "
            f"{len(high_corr_pairs_products)} pares con correlación > 0.7, "
            f"{high_corr_count} pares con correlación > 0.95"
        )

    def drop_high_correlated_cols(self):
        """Elimina columnas altamente correlacionadas, conservando las más relevantes."""
        logger.info("Iniciando eliminación de columnas altamente correlacionadas")
        
        if self.high_corr_df_products is None:
            raise ValueError("Debe ejecutar get_high_pair_correlations primero")
        
        previous_shape = self.processed_df.shape
        initial_cols = len(self.processed_df.columns)
        
        high_corr_filtered = self.high_corr_df_products[self.high_corr_df_products['correlation'] > 0.95]
        
        for _, row in high_corr_filtered.iterrows():
            var1 = row['var1']
            var2 = row['var2']

            # Comparar la correlación absoluta con el target
            corr_var1 = abs(self.cor_target[var1])
            corr_var2 = abs(self.cor_target[var2])

            # Quedarse con la columna más correlacionada, eliminar la otra
            if corr_var1 < corr_var2:
                col_to_drop = var1
                col_to_keep = var2
            else:
                col_to_drop = var2
                col_to_keep = var1
            
            if col_to_drop not in self.cols_to_drop_:
                self.cols_to_drop_.append(col_to_drop)
                logger.debug(f"Marcada para eliminar: {col_to_drop} (corr: {corr_var1:.3f}) "
                           f"vs {col_to_keep} (corr: {corr_var2:.3f})")

        self.cols_to_drop_ = list(set(self.cols_to_drop_))
        self.product_cols = list(set(product_cols)-set(self.cols_to_drop_))
        
        # Eliminar columnas
        columns_before_drop = set(self.processed_df.columns)
        self.processed_df = self.processed_df.drop(columns=self.cols_to_drop_)
        columns_after_drop = set(self.processed_df.columns)
        dropped_columns = columns_before_drop - columns_after_drop
        
        final_cols = len(self.processed_df.columns)
        
        logger.info(
            f"Eliminación de columnas correlacionadas completada - "
            f"{len(dropped_columns)} columnas eliminadas: {list(dropped_columns)}"
        )
        self._log_shape_change(previous_shape, "Eliminación de columnas correlacionadas")

    def correct_skewness(self):
        """
        Corrige asimetría en las columnas usando transformación Yeo-Johnson.
        
        Args:
            product_cols: Lista de columnas de productos a transformar
        """
        logger.info("Iniciando corrección de asimetría")
        
        # Calcular asimetría inicial
        skewness_before = self.processed_df[self.product_cols].skew()
        self.skewed_cols_ = skewness_before[abs(skewness_before) > 0.5].index.tolist()
        
        if not self.skewed_cols_:
            logger.info("No se encontraron columnas con asimetría significativa (> 0.5)")
            return
            
        logger.info(f"{len(self.skewed_cols_)} columnas con asimetría > 0.5: {self.skewed_cols_}")
        
        # Aplicar transformación Yeo-Johnson
        pt = PowerTransformer(method='yeo-johnson')
        self.processed_df[self.skewed_cols_] = pt.fit_transform(self.processed_df[self.skewed_cols_])
        
        # Calcular asimetría después de la transformación
        skewness_after = self.processed_df[self.skewed_cols_].skew()
        
        # Guardar parámetros
        if hasattr(pt, "lambdas_"):
            self.power_params_ = dict(zip(self.skewed_cols_, pt.lambdas_))
        else:
            self.power_params_ = {}
        
        logger.info(
            f"Corrección de asimetría completada - "
            f"Asimetría promedio: {skewness_before.mean():.3f} → {skewness_after.mean():.3f}"
        )

    def apply_one_hot(self):
        """Aplica one-hot encoding a la columna 'zone'."""
        logger.info("Aplicando one-hot encoding a la columna 'zone'")
        
        if 'zone' not in self.processed_df.columns:
            raise ValueError("Columna 'zone' no encontrada para one-hot encoding")
        
        previous_shape = self.processed_df.shape
        
        # Convertir a entero y aplicar one-hot
        self.processed_df['zone'] = self.processed_df['zone'].astype(int)
        zone_dummies = pd.get_dummies(self.processed_df['zone'], prefix='zone')
        
        # Concatenar y eliminar columna original
        self.processed_df = pd.concat([
            self.processed_df.drop('zone', axis=1), 
            zone_dummies
        ], axis=1)
        
        logger.info(f"One-hot encoding completado - {len(zone_dummies.columns)} columnas zone creadas")
        self._log_shape_change(previous_shape, "One-hot encoding")

    def apply_preprocess(self, sociodemographic_cols: list, product_cols: list) -> pd.DataFrame:
        """
        Ejecuta el pipeline completo de preprocesamiento.
        
        Args:
            sociodemographic_cols: Columnas sociodemográficas a agrupar
            product_cols: Columnas de productos para análisis de correlación
            
        Returns:
            DataFrame procesado
        """
        logger.info("=== INICIANDO PIPELINE COMPLETO DE PREPROCESAMIENTO ===")
        logger.info(f"Columnas sociodemográficas: {sociodemographic_cols}")
        logger.info(f"Columnas de productos: {product_cols}")
        
        # Pipeline de procesamiento
        self.group_sociodemographic_cols(sociodemographic_cols)
        self.remove_duplicates()
        self.handle_complex_duplicates()
        self.get_correlations()
        self.get_high_pair_correlations(product_cols)
        self.drop_high_correlated_cols()
        self.correct_skewness()
        self.apply_one_hot()
        
        logger.info("=== PIPELINE COMPLETADO EXITOSAMENTE ===")
        logger.info(f"Shape final del DataFrame: {self.processed_df.shape}")
        logger.info(f"Columnas eliminadas: {len(self.cols_to_drop_)}")
        logger.info(f"Columnas transformadas: {len(self.skewed_cols_)}")
        
        return self.processed_df

    def get_preprocessing_summary(self) -> dict:
        """Retorna un resumen del proceso de preprocesamiento."""
        return {
            'original_shape': self.original_df.shape,
            'processed_shape': self.processed_df.shape if self.processed_df is not None else None,
            'zones_created': len(self.zone_mapper_df) if self.zone_mapper_df is not None else 0,
            'columns_dropped': len(self.cols_to_drop_),
            'columns_skewness_corrected': len(self.skewed_cols_),
            'high_correlation_pairs': len(self.high_corr_df_products) if self.high_corr_df_products is not None else 0
        }

In [3]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
import logging
from typing import Union, Tuple, Optional, Dict, Any
import numpy as np

class H2OAutoMLTrainer:
    def __init__(self, max_models: int = 20, max_runtime_secs: int = 300, 
                 seed: int = 42, balance_classes: bool = True, 
                 stopping_metric: str = "AUC"):
        """
        Inicializa el entrenador de H2O AutoML.
        
        Args:
            max_models: Número máximo de modelos a entrenar
            max_runtime_secs: Tiempo máximo de ejecución en segundos
            seed: Semilla para reproducibilidad
            balance_classes: Balancear clases para datos desbalanceados
            stopping_metric: Métrica para early stopping
        """
        self.max_models = max_models
        self.max_runtime_secs = max_runtime_secs
        self.seed = seed
        self.balance_classes = balance_classes
        self.stopping_metric = stopping_metric
        
        # Atributos del modelo
        self.aml = None
        self.leader = None
        self.leaderboard = None
        self.h2o_frame = None
        
        # Estado de H2O
        self.h2o_initialized = False
        
        logger.info(f"H2OAutoMLTrainer inicializado - max_models: {max_models}, "
                   f"max_runtime_secs: {max_runtime_secs}")

    def initialize_h2o(self) -> None:
        """Inicializa la conexión con H2O."""
        if not self.h2o_initialized:
            try:
                h2o.init()
                self.h2o_initialized = True
                logger.info("H2O inicializado exitosamente")
            except Exception as e:
                logger.error(f"Error inicializando H2O: {e}")
                raise

    def prepare_data(self, df: pd.DataFrame, target_col: str = 'target') -> None:
        """
        Prepara los datos para H2O AutoML.
        
        Args:
            df: DataFrame con los datos
            target_col: Nombre de la columna target
        """
        logger.info(f"Preparando datos - Shape: {df.shape}, Target: {target_col}")
        
        if not self.h2o_initialized:
            self.initialize_h2o()
        
        # Convertir a H2OFrame
        self.h2o_frame = h2o.H2OFrame(df)
        
        # Convertir target a factor (para clasificación)
        if target_col in self.h2o_frame.columns:
            self.h2o_frame[target_col] = self.h2o_frame[target_col].asfactor()
            logger.info(f"Target '{target_col}' convertido a factor")
        
        logger.info(f"Datos preparados - H2OFrame shape: {self.h2o_frame.shape}")

    def train(self, target_col: str = 'target', 
              training_frame: Optional[h2o.H2OFrame] = None) -> None:
        """
        Entrena el modelo AutoML.
        
        Args:
            target_col: Columna target
            training_frame: Frame de entrenamiento (opcional, usa self.h2o_frame por defecto)
        """
        logger.info("Iniciando entrenamiento AutoML")
        
        if training_frame is None:
            if self.h2o_frame is None:
                raise ValueError("No hay datos preparados. Ejecute prepare_data primero.")
            training_frame = self.h2o_frame
        
        # Configurar AutoML
        self.aml = H2OAutoML(
            max_models=self.max_models,
            seed=self.seed,
            max_runtime_secs=self.max_runtime_secs,
            balance_classes=self.balance_classes,
            stopping_metric=self.stopping_metric
        )
        
        # Entrenar
        self.aml.train(y=target_col, training_frame=training_frame)
        
        # Obtener resultados
        self.leader = self.aml.leader
        self.leaderboard = self.aml.leaderboard
        
        logger.info("Entrenamiento AutoML completado exitosamente")
        self._log_training_summary()

    def _log_training_summary(self) -> None:
        """Registra un resumen del entrenamiento."""
        if self.leaderboard is not None:
            lb_head = self.leaderboard.head()
            logger.info("Leaderboard - Top modelos:")
            for i, (model_id, *metrics) in enumerate(lb_head.as_data_frame().itertuples(index=False)):
                logger.info(f"  {i+1}. {model_id} - {metrics}")
        
        if self.leader is not None:
            logger.info(f"Modelo líder: {self.leader.model_id}")

    def predict(self, X_data: Union[pd.DataFrame, h2o.H2OFrame], 
                best_threshold: Optional[float] = None,
                auto_threshold: bool = False,
                y_true: Optional[np.ndarray] = None,
                return_h2o_frame: bool = False) -> Union[pd.DataFrame, h2o.H2OFrame]:
        """
        Realiza predicciones con el modelo líder.

        Args:
            X_data: Datos para predecir
            best_threshold: Umbral óptimo para clasificación (opcional)
            auto_threshold: Si calcular automáticamente el threshold que maximiza AUC
            y_true: Valores reales (requerido si auto_threshold=True)
            return_h2o_frame: Si retornar H2OFrame en lugar de DataFrame

        Returns:
            DataFrame o H2OFrame con predicciones
        """
        logger.info("Realizando predicciones")

        if self.leader is None:
            raise ValueError("No hay modelo entrenado. Ejecute train primero.")

        # Validar parámetros para auto_threshold
        if auto_threshold and y_true is None:
            raise ValueError("y_true es requerido cuando auto_threshold=True")

        # Convertir a H2OFrame si es necesario
        if isinstance(X_data, pd.DataFrame):
            h2o_X = h2o.H2OFrame(X_data)
            logger.info(f"DataFrame convertido a H2OFrame - Shape: {h2o_X.shape}")
        else:
            h2o_X = X_data

        # Realizar predicciones
        predictions = self.leader.predict(h2o_X)

        if return_h2o_frame:
            # Combinar features con predicciones
            results_h2o = h2o_X.cbind(predictions)
            logger.info("Predicciones completadas - Retornando H2OFrame")
            return results_h2o
        else:
            # Convertir a DataFrame de pandas
            features_df = h2o_X.as_data_frame().copy()
            probabilities = predictions['p1'].as_data_frame().values.flatten()
            predicted_classes = predictions['predict'].as_data_frame().values.flatten()

            results_df = features_df.copy()
            results_df['predicted_prob'] = probabilities
            results_df['predicted_class'] = predicted_classes

            # Calcular threshold automáticamente si se solicita
            if auto_threshold:
                from sklearn.metrics import roc_curve, auc

                # Calcular curva ROC
                fpr, tpr, thresholds = roc_curve(y_true, probabilities)
                roc_auc = auc(fpr, tpr)

                # Encontrar threshold que maximiza AUC (punto más alejado de la diagonal)
                optimal_idx = np.argmax(tpr - fpr)
                best_threshold = thresholds[optimal_idx]

                logger.info(f"Threshold automático calculado: {best_threshold:.4f} (AUC: {roc_auc:.4f})")

            # Aplicar threshold si se proporciona o se calcula automáticamente
            if best_threshold is not None:
                logger.info(f"Umbral aplicado: {best_threshold}")

                results_df['prediction'] = ((results_df['predicted_prob'] >= best_threshold) & 
                                            (results_df['predicted_class'] == 1)).astype(int)
            else:
                # Si no hay threshold, usar predicted_class directamente
                results_df['prediction'] = results_df['predicted_class']

            logger.info(f"Predicciones completadas - DataFrame shape: {results_df.shape}")
            return results_df

    def predict_with_actuals(self, X_data: Union[pd.DataFrame, h2o.H2OFrame], 
                           y_actual: pd.Series,
                           best_threshold: Optional[float] = None) -> pd.DataFrame:
        """
        Realiza predicciones e incluye los valores reales.
        
        Args:
            X_data: Features para predecir
            y_actual: Valores reales del target
            best_threshold: Umbral óptimo para clasificación
            
        Returns:
            DataFrame con predicciones y valores reales
        """
        logger.info("Realizando predicciones con valores reales")
        
        # Obtener predicciones base
        results_df = self.predict(X_data, best_threshold=best_threshold)
        
        # Agregar valores reales
        results_df['actual'] = y_actual.values
        
        logger.info(f"Predicciones con valores reales completadas - Shape: {results_df.shape}")
        return results_df

    def get_model_performance(self, test_data: Optional[h2o.H2OFrame] = None, 
                            target_col: str = 'target') -> Any:
        """
        Evalúa el performance del modelo líder.
        
        Args:
            test_data: Datos de test (opcional)
            target_col: Columna target
            
        Returns:
            Objeto de performance del modelo
        """
        logger.info("Evaluando performance del modelo")
        
        if self.leader is None:
            raise ValueError("No hay modelo entrenado. Ejecute train primero.")
        
        if test_data is None:
            if self.h2o_frame is None:
                raise ValueError("No hay datos disponibles para evaluación")
            test_data = self.h2o_frame
        
        performance = self.leader.model_performance(test_data=test_data)
        
        # Log de métricas principales
        if hasattr(performance, 'auc'):
            logger.info(f"AUC: {performance.auc():.4f}")
        if hasattr(performance, 'logloss'):
            logger.info(f"LogLoss: {performance.logloss():.4f}")
        if hasattr(performance, 'accuracy'):
            logger.info(f"Accuracy: {performance.accuracy():.4f}")
        
        return performance

    def get_leaderboard(self, as_dataframe: bool = True) -> Union[h2o.H2OFrame, pd.DataFrame]:
        """
        Obtiene el leaderboard de modelos.
        
        Args:
            as_dataframe: Si retornar como DataFrame de pandas
            
        Returns:
            Leaderboard de modelos
        """
        if self.leaderboard is None:
            raise ValueError("No hay leaderboard disponible. Ejecute train primero.")
        
        if as_dataframe:
            return self.leaderboard.as_data_frame()
        else:
            return self.leaderboard

    def save_model(self, path: str, model_id: Optional[str] = None) -> None:
        """
        Guarda el modelo en disco.
        
        Args:
            path: Ruta donde guardar el modelo
            model_id: ID del modelo a guardar (None para el líder)
        """
        if model_id is None:
            if self.leader is None:
                raise ValueError("No hay modelo líder para guardar")
            model = self.leader
        else:
            model = h2o.get_model(model_id)
        
        model_path = h2o.save_model(model=model, path=path, force=True)
        logger.info(f"Modelo guardado en: {model_path}")

    def save_model(self, model_name: str = "h2o_automl_model", base_path: str = "./") -> str:
        """
        Guarda el modelo en disco con nombre configurable.

        Args:
            model_name: Nombre del modelo (sin extensión)
            base_path: Ruta base donde guardar el modelo

        Returns:
            Ruta completa donde se guardó el modelo
        """
        if self.leader is None:
            raise ValueError("No hay modelo líder para guardar")

        # Crear directorio si no existe
        os.makedirs(base_path, exist_ok=True)

        # Guardar modelo
        model_path = h2o.save_model(
            model=self.leader, 
            path=base_path, 
            force=True,
            filename=model_name
        )

        logger.info(f"Modelo guardado en: {model_path}")

    def shutdown_h2o(self) -> None:
        """Cierra la conexión con H2O."""
        if self.h2o_initialized:
            h2o.cluster().shutdown()
            self.h2o_initialized = False
            logger.info("H2O shutdown completado")

    def get_training_summary(self) -> Dict[str, Any]:
        """
        Retorna un resumen del entrenamiento.
        
        Returns:
            Diccionario con información del entrenamiento
        """
        summary = {
            'max_models': self.max_models,
            'max_runtime_secs': self.max_runtime_secs,
            'seed': self.seed,
            'balance_classes': self.balance_classes,
            'stopping_metric': self.stopping_metric,
            'leader_model': self.leader.model_id if self.leader else None,
            'models_trained': len(self.leaderboard) if self.leaderboard else 0,
            'h2o_initialized': self.h2o_initialized
        }
        
        if self.leaderboard is not None and len(self.leaderboard) > 0:
            lb_df = self.leaderboard.as_data_frame()
            summary['top_model_auc'] = lb_df.iloc[0]['auc'] if 'auc' in lb_df.columns else None
        
        return summary

    def __enter__(self):
        """Support for context manager."""
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Cleanup on context manager exit."""
        self.shutdown_h2o()




In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

file_name = '../data/raw/insurance_company_original.csv'
sociodemographic_cols = [f"SD_{i}" for i in range(1, 44)]
product_cols = [f"PD_{i-44}" for i in range(44, 86)]
cols = sociodemographic_cols + product_cols + ["target"]

df = pd.read_csv(file_name, header=None, names=cols)
df = df.iloc[1:].reset_index(drop=True) # Viene del analisis



preprocessor = Preprocessor(df)
processed = preprocessor.apply_preprocess(sociodemographic_cols, product_cols)
trainer = H2OAutoMLTrainer(max_models=20, max_runtime_secs=300)
trainer.prepare_data(processed, target_col='target')
trainer.train(target_col='target')

X = processed.drop(columns='target')
y = processed['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42  # stratify para clasificación
)


predictions = trainer.predict(X_test, auto_threshold=True, y_true=y_test)



2025-11-08 03:10:00,293 - preprocessor_logger - INFO - [__init__] - Preprocessor inicializado - Shape original: (5821, 86)
2025-11-08 03:10:00,294 - preprocessor_logger - INFO - [apply_preprocess] - === INICIANDO PIPELINE COMPLETO DE PREPROCESAMIENTO ===
2025-11-08 03:10:00,294 - preprocessor_logger - INFO - [apply_preprocess] - Columnas sociodemográficas: ['SD_1', 'SD_2', 'SD_3', 'SD_4', 'SD_5', 'SD_6', 'SD_7', 'SD_8', 'SD_9', 'SD_10', 'SD_11', 'SD_12', 'SD_13', 'SD_14', 'SD_15', 'SD_16', 'SD_17', 'SD_18', 'SD_19', 'SD_20', 'SD_21', 'SD_22', 'SD_23', 'SD_24', 'SD_25', 'SD_26', 'SD_27', 'SD_28', 'SD_29', 'SD_30', 'SD_31', 'SD_32', 'SD_33', 'SD_34', 'SD_35', 'SD_36', 'SD_37', 'SD_38', 'SD_39', 'SD_40', 'SD_41', 'SD_42', 'SD_43']
2025-11-08 03:10:00,294 - preprocessor_logger - INFO - [apply_preprocess] - Columnas de productos: ['PD_0', 'PD_1', 'PD_2', 'PD_3', 'PD_4', 'PD_5', 'PD_6', 'PD_7', 'PD_8', 'PD_9', 'PD_10', 'PD_11', 'PD_12', 'PD_13', 'PD_14', 'PD_15', 'PD_16', 'PD_17', 'PD_18', '

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "23.0.2" 2025-01-21; OpenJDK Runtime Environment Homebrew (build 23.0.2); OpenJDK 64-Bit Server VM Homebrew (build 23.0.2, mixed mode, sharing)
  Starting server from /Users/luis.caporal/miniconda3/envs/notebooks/lib/python3.11/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/v4/2y3d_dfd7hv8b31yqy3186v40000gp/T/tmpu0mh8rlv
  JVM stdout: /var/folders/v4/2y3d_dfd7hv8b31yqy3186v40000gp/T/tmpu0mh8rlv/h2o_luis_caporal_started_from_python.out
  JVM stderr: /var/folders/v4/2y3d_dfd7hv8b31yqy3186v40000gp/T/tmpu0mh8rlv/h2o_luis_caporal_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/Mexico_City
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.8
H2O_cluster_version_age:,30 days
H2O_cluster_name:,H2O_from_python_luis_caporal_ugmo7o
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.977 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


2025-11-08 03:10:07,812 - preprocessor_logger - INFO - [initialize_h2o] - H2O inicializado exitosamente


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


...

2025-11-08 03:13:25,695 - preprocessor_logger - INFO - [train] - Entrenamiento AutoML completado exitosamente
2025-11-08 03:13:25,738 - preprocessor_logger - INFO - [_log_training_summary] - Leaderboard - Top modelos:
2025-11-08 03:13:25,779 - preprocessor_logger - INFO - [_log_training_summary] -   1. StackedEnsemble_BestOfFamily_1_AutoML_1_20251108_31013 - [0.8101541043637474, 0.1900508170917577, 0.2668104853254295, 0.3636328959414098, 0.2256458093010703, 0.0509160312551349]
2025-11-08 03:13:25,779 - preprocessor_logger - INFO - [_log_training_summary] -   2. GBM_1_AutoML_1_20251108_31013 - [0.8067973756484589, 0.1940447780083969, 0.2163126517039348, 0.3446368629844369, 0.2275361231030927, 0.0517726873167857]
2025-11-08 03:13:25,779 - preprocessor_logger - INFO - [_log_training_summary] -   3. XGBoost_1_AutoML_1_20251108_31013 - [0.8043942630454685, 0.5397874427693786, 0.2543667701638603, 0.3112602990540128, 0.4331105107301356, 0.1875847145049188]
2025-11-08 03:13:25,780 - preprocess

Parse progress: |

2025-11-08 03:13:26,550 - preprocessor_logger - INFO - [predict] - DataFrame convertido a H2OFrame - Shape: (1025, 1769)


████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


2025-11-08 03:13:27,245 - preprocessor_logger - INFO - [predict] - Threshold automático calculado: 0.0539 (AUC: 0.8591)
2025-11-08 03:13:27,245 - preprocessor_logger - INFO - [predict] - Umbral aplicado: 0.0538524265962822
2025-11-08 03:13:27,246 - preprocessor_logger - INFO - [predict] - Predicciones completadas - DataFrame shape: (1025, 1772)


In [5]:
results_with_actuals = trainer.predict_with_actuals(X_test, y_test, best_threshold=0.0528524669190337)

trainer.save_model(model_name="h2o_automl_model")


2025-11-08 03:13:27,250 - preprocessor_logger - INFO - [predict_with_actuals] - Realizando predicciones con valores reales
2025-11-08 03:13:27,250 - preprocessor_logger - INFO - [predict] - Realizando predicciones


Parse progress: |

2025-11-08 03:13:27,929 - preprocessor_logger - INFO - [predict] - DataFrame convertido a H2OFrame - Shape: (1025, 1769)


████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


2025-11-08 03:13:28,575 - preprocessor_logger - INFO - [predict] - Umbral aplicado: 0.0528524669190337
2025-11-08 03:13:28,576 - preprocessor_logger - INFO - [predict] - Predicciones completadas - DataFrame shape: (1025, 1772)
2025-11-08 03:13:28,578 - preprocessor_logger - INFO - [predict_with_actuals] - Predicciones con valores reales completadas - Shape: (1025, 1773)
2025-11-08 03:13:28,671 - preprocessor_logger - INFO - [save_model] - Modelo guardado en: /Users/luis.caporal/Documents/Master - TEC/9. MLOps/MNA_MLOps_Equipo12/notebooks/h2o_automl_model


In [6]:
results_with_actuals[(results_with_actuals['actual']==1)
                    &(results_with_actuals['prediction']==1)]

PD_0      PD_1      PD_2  PD_3      PD_4      PD_5      PD_6      PD_7  \
71       1 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
278      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
295      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
302      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
362      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
373      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
409      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
427      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
455      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
544      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
551      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
573      0 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
667      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
668      2 -0.127566 -0.154904     6 -0.097272  4.719881 -0.041959 -0.113384   
675      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
705      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
710      3 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
730      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
769      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
924      0 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
928      2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
976      0 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   
1012     2 -0.127566 -0.154904     6 -0.097272 -0.211870 -0.041959 -0.113384   

          PD_8      PD_9     PD_11     PD_12     PD_13     PD_14  PD_15  \
71   -0.169489 -0.064169  4.074108 -0.078041 -0.086463 -0.067168      4   
278  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
295  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
302  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
362  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
373   5.900101 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
409  -0.169489 -0.064169  4.074108 -0.078041 -0.086463 -0.067168      4   
427  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
455  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
544  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
551  -0.169489 -0.064169  4.074108 -0.078041 -0.086463 -0.067168      3   
573  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
667  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
668  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
675  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
705  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
710  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
730  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
769  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      3   
924  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
928  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   
976  -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      0   
1012 -0.169489 -0.064169 -0.245456 -0.078041 -0.086463 -0.067168      4   

         PD_16      PD_17     PD_18     PD_19     PD_22     PD_24     PD_25  \
71   -0.024211  -0.080535 -0.170704 -0.094155 -0.127566  0.873617 -0.097272   
278  -0.024211  -0.080535 -0.170704 -0.094155 -0.127566  0.873617 -0.097272   
295  -0.024211  -0.080535 -0.170704 -0.094155 -

In [7]:
preprocessor.zones_

['1435',
 '1434',
 '415',
 '1680',
 '746',
 '1651',
 '1126',
 '1137',
 '551',
 '483',
 '404',
 '1130',
 '1704',
 '729',
 '1506',
 '721',
 '627',
 '1027',
 '1255',
 '1266',
 '603',
 '1306',
 '634',
 '1190',
 '1463',
 '1669',
 '1058',
 '1261',
 '854',
 '766',
 '1257',
 '1505',
 '280',
 '239',
 '1714',
 '1610',
 '1263',
 '851',
 '543',
 '300',
 '1143',
 '144',
 '1536',
 '1392',
 '1562',
 '1636',
 '1265',
 '1140',
 '868',
 '676',
 '514',
 '1268',
 '1656',
 '497',
 '562',
 '1313',
 '999',
 '987',
 '1414',
 '1287',
 '788',
 '498',
 '1251',
 '1407',
 '612',
 '1426',
 '961',
 '435',
 '890',
 '1107',
 '352',
 '522',
 '145',
 '1502',
 '506',
 '85',
 '615',
 '77',
 '102',
 '29',
 '1711',
 '744',
 '1548',
 '25',
 '1645',
 '1367',
 '1494',
 '406',
 '349',
 '152',
 '898',
 '1583',
 '535',
 '1578',
 '369',
 '1396',
 '813',
 '1164',
 '32',
 '550',
 '1478',
 '853',
 '59',
 '768',
 '1429',
 '344',
 '1307',
 '1200',
 '724',
 '1275',
 '1172',
 '268',
 '1204',
 '1480',
 '1364',
 '399',
 '454',
 '982',
 '14

In [8]:
preprocessor.zone_mapper_df


,SD_1,SD_2,SD_3,SD_4,SD_5,SD_6,SD_7,SD_8,SD_9,SD_10,SD_11,SD_12,SD_13,SD_14,SD_15,SD_16,SD_17,SD_18,SD_19,SD_20,SD_21,SD_22,SD_23,SD_24,SD_25,SD_26,SD_27,SD_28,SD_29,SD_30,SD_31,SD_32,SD_33,SD_34,SD_35,SD_36,SD_37,SD_38,SD_39,SD_40,SD_41,SD_42,SD_43,zone
0,37,1,2,2,8,1,4,1.0,4.0,6,2.0,2.0,0.0,4.0,5,0.0,5.0,4.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,2.0,3.0,5.0,0.0,2.0,7.0,7.0,1.0,2.0,6.0,3.0,2.0,0,5.0,2.0,0.0,5.0,4.0,1435
1,37,1,2,2,8,0,4,2.0,4.0,3,2.0,4.0,4.0,4.0,2,0.0,5.0,4.0,0.0,0.0,0.0,7.0,0.0,2.0,0.0,5.0,0.0,4.0,0.0,7.0,2.0,7.0,0.0,2.0,9.0,0.0,4.0,5,0.0,0.0,0.0,3.0,4.0,1434
2,9,1,3,3,3,2,3,2.0,4.0,5,2.0,2.0,2.0,3.0,4,3.0,4.0,2.0,4.0,0.0,0.0,3.0,1.0,2.0,3.0,2.0,1.0,4.0,0.0,5.0,4.0,9.0,0.0,0.0,7.0,2.0,1.0,5,3.0,0.0,0.0,4.0,4.0,415
3,40,1,4,2,10,1,4,1.0,4.0,7,1.0,2.0,2.0,4.0,4,5.0,4.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,4.0,5.0,6.0,2.0,1.0,5.0,4.0,0.0,0,9.0,0.0,0.0,6.0,3.0,1680
4,23,1,2,1,5,0,5,0.0,5.0,0,6.0,3.0,3.0,5.0,2,0.0,5.0,4.0,2.0,0.0,0.0,4.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,9.0,0.0,5.0,3.0,3.0,9.0,0.0,5.0,2,3.0,0.0,0.0,3.0,3.0,746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5785,41,1,2,3,10,0,9,0.0,0.0,9,0.0,0.0,3.0,5.0,2,0.0,6.0,3.0,0.0,0.0,0.0,6.0,2.0,2.0,1.0,2.0,2.0,2.0,3.0,4.0,5.0,9.0,0.0,0.0,7.0,2.0,3.0,2,4.0,0.0,0.0,3.0,5.0,1690
5797,27,1,1,6,6,1,5,2.0,3.0,5,1.0,3.0,5.0,4.0,1,4.0,5.0,1.0,5.0,0.0,0.0,4.0,0.0,0.0,5.0,4.0,1.0,0.0,1.0,8.0,1.0,6.0,1.0,3.0,7.0,2.0,5.0,4,0.0,0.0,0.0,3.0,1.0,920
5801,3,1,2,4,1,1,4,1.0,4.0,8,0.0,1.0,2.0,5.0,3,3.0,3.0,4.0,5.0,1.0,0.0,2.0,1.0,1.0,3.0,2.0,2.0,3.0,1.0,3.0,6.0,6.0,3.0,1.0,3.0,6.0,2.0,1,4.0,2.0,2.0,7.0,6.0,103
5810,39,1,3,2,9,1,5,1.0,3.0,7,1.0,2.0,2.0,3.0,5,1.0,5.0,3.0,3.0,0.0,3.0,1.0,2.0,2.0,3.0,2.0,3.0,3.0,1.0,3.0,6.0,5.0,3.0,1.0,4.0,5.0,3.0,3,3.0,1.0,0.0,4.0,5.0,1613


In [9]:
preprocessor.cols_to_drop_


['PD_23', 'PD_34', 'PD_21', 'PD_28', 'PD_10', 'PD_20']

In [10]:
preprocessor.power_params_

{'PD_19': -109.10091367166753,
 'PD_9': -162.95710298303104,
 'PD_2': -28.627988106034252,
 'PD_7': -76.85032119245798,
 'PD_36': 0.14173046150032026,
 'PD_33': -238.3239198773984,
 'PD_18': -50.952009135765294,
 'PD_22': -88.38960113285489,
 'PD_27': -661.4226229132257,
 'PD_29': -43.23134439097251,
 'PD_11': -11.395191327923461,
 'PD_8': -23.208224470467147,
 'PD_25': -135.15918496733266,
 'PD_16': -1847.0102988240042,
 'PD_13': -112.58128690002786,
 'PD_24': -0.4935977481602459,
 'PD_4': -55.5045099116365,
 'PD_30': -260.9036058585604,
 'PD_6': -292.0712253885379,
 'PD_31': -20.06562294975205,
 'PD_35': -291.55820594209365,
 'PD_38': -216.21474660038908,
 'PD_12': -135.07763171284992,
 'PD_41': -89.45988607730936,
 'PD_37': -2462.6804341422626,
 'PD_39': -44.26926660334587,
 'PD_40': -162.07189988610938,
 'PD_17': -110.76213740576299,
 'PD_14': -114.9949962165707,
 'PD_5': -13.59551449931869,
 'PD_32': -19.856328792623778,
 'PD_1': -48.025428714193346,
 'PD_26': -32.40091196720402}

In [11]:
preprocessor.skewed_cols_

['PD_19',
 'PD_9',
 'PD_2',
 'PD_7',
 'PD_36',
 'PD_33',
 'PD_18',
 'PD_22',
 'PD_27',
 'PD_29',
 'PD_11',
 'PD_8',
 'PD_25',
 'PD_16',
 'PD_13',
 'PD_24',
 'PD_4',
 'PD_30',
 'PD_6',
 'PD_31',
 'PD_35',
 'PD_38',
 'PD_12',
 'PD_41',
 'PD_37',
 'PD_39',
 'PD_40',
 'PD_17',
 'PD_14',
 'PD_5',
 'PD_32',
 'PD_1',
 'PD_26']

In [12]:

class SociodemographicToZoneTransformer(BaseEstimator):
    def __init__(self, sociodemographic_cols, zone_mapper_df):
        """
        Transforma columnas sociodemográficas a zona y aplica one-hot encoding.
        
        Args:
            sociodemographic_cols: Lista de columnas sociodemográficas
            zone_mapper_df: DataFrame con mapeo sociodemográfico -> zona
        """
        self.sociodemographic_cols = sociodemographic_cols
        self.zone_mapper_df = zone_mapper_df
        self.available_zones_ = None
        
    def fit(self, X, y=None):
        logger.info("Fitteando SociodemographicToZoneTransformer")
        # Determinar las zonas disponibles del mapper
        self.available_zones_ = sorted(self.zone_mapper_df['zone'].unique())
        logger.info(f"Zonas disponibles: {self.available_zones_}")
        return self
    
    def transform(self, X):
        logger.info("Transformando columnas sociodemográficas a zona")
        X_transformed = X.copy()
        
        # 1. Mapear a zona
        X_transformed = X_transformed.merge(
            self.zone_mapper_df,
            on=self.sociodemographic_cols,
            how='left'
        )
        
        # Verificar zonas no mapeadas
        zonas_nulas = X_transformed['zone'].isna().sum()
        if zonas_nulas > 0:
            logger.warning(f"{zonas_nulas} registros sin zona asignada")
            # Rellenar con zona más frecuente o valor por defecto
            X_transformed['zone'] = X_transformed['zone'].fillna(self.available_zones_[0])
        
        # 2. Aplicar one-hot encoding
        zone_dummies = pd.get_dummies(X_transformed['zone'], prefix='zone')
        
        # Asegurar que tenemos todas las zonas esperadas
        for zone in self.available_zones_:
            col_name = f'zone_{zone}'
            if col_name not in zone_dummies.columns:
                zone_dummies[col_name] = 0
        
        # 3. Eliminar columnas originales
        columns_to_drop = self.sociodemographic_cols + ['zone']
        X_transformed = X_transformed.drop(columns=columns_to_drop)
        
        # 4. Concatenar con one-hot encoding
        X_transformed = pd.concat([X_transformed, zone_dummies], axis=1)
        
        logger.info(f"Transformación completada. Shape: {X_transformed.shape}")
        return X_transformed


class ColumnDropper(BaseEstimator):
    def __init__(self, columns_to_drop):
        """
        Elimina columnas especificadas del DataFrame.
        
        Args:
            columns_to_drop: Lista de columnas a eliminar
        """
        self.columns_to_drop = columns_to_drop
    
    def fit(self, X, y=None):
        logger.info("Fitteando ColumnDropper")
        # Verificar qué columnas existen realmente
        self.existing_columns_to_drop_ = [
            col for col in self.columns_to_drop if col in X.columns
        ]
        logger.info(f"Columnas a eliminar: {self.existing_columns_to_drop_}")
        return self
    
    def transform(self, X):
        logger.info("Eliminando columnas especificadas")
        X_transformed = X.copy()
        
        if self.existing_columns_to_drop_:
            X_transformed = X_transformed.drop(columns=self.existing_columns_to_drop_)
            logger.info(f"Eliminadas {len(self.existing_columns_to_drop_)} columnas")
        
        logger.info(f"Transformación completada. Shape: {X_transformed.shape}")
        return X_transformed


class SkewnessCorrector(BaseEstimator):
    def __init__(self, power_params):
        """
        Aplica corrección de asimetría usando parámetros predefinidos.
        
        Args:
            power_params: Diccionario {columna: lambda} para transformación
        """
        self.power_params = power_params
    
    def fit(self, X, y=None):
        logger.info("Fitteando SkewnessCorrector")
        # Verificar qué columnas existen
        self.existing_power_params_ = {
            col: lambda_val for col, lambda_val in self.power_params.items() 
            if col in X.columns
        }
        logger.info(f"Columnas para corrección: {list(self.existing_power_params_.keys())}")
        return self
    
    def transform(self, X):
        logger.info("Aplicando corrección de asimetría")
        X_transformed = X.copy()
        
        for col, lambda_val in self.existing_power_params_.items():
            # Aplicar transformación Yeo-Johnson
            if lambda_val == 0:
                # Log transformation
                X_transformed[col] = np.log1p(X_transformed[col])
            elif lambda_val == 1:
                # No transformation needed
                pass
            else:
                # General Yeo-Johnson transformation
                X_transformed[col] = ((X_transformed[col] + 1) ** lambda_val - 1) / lambda_val
            
            logger.debug(f"Transformada columna {col} con lambda: {lambda_val}")
        
        logger.info("Corrección de asimetría completada")
        return X_transformed


class H2OPredictor(BaseEstimator):
    def __init__(self, model_path, best_threshold=None):
        """
        Realiza predicciones usando modelo H2O.
        
        Args:
            model_path: Ruta al modelo H2O guardado
            best_threshold: Umbral para clasificación (opcional)
        """
        self.model_path = model_path
        self.best_threshold = best_threshold
        self.h2o_model = None
        self.h2o_initialized = False
        
    def _initialize_h2o(self):
        """Inicializa H2O si no está inicializado."""
        if not self.h2o_initialized:
            h2o.init()
            self.h2o_initialized = True
            logger.info("H2O inicializado")
    
    def fit(self, X, y=None):
        logger.info("Fitteando H2OPredictor")
        self._initialize_h2o()
        
        # Cargar modelo H2O
        self.h2o_model = h2o.load_model(self.model_path)
        logger.info(f"Modelo H2O cargado: {self.h2o_model.model_id}")
        
        return self
    
    def transform(self, X):
        logger.info("Realizando predicciones con H2O")
        X_transformed = X.copy()
        
        # Convertir a H2OFrame
        h2o_df = h2o.H2OFrame(X_transformed)
        
        # Realizar predicciones
        predictions = self.h2o_model.predict(h2o_df)
        
        # Extraer probabilidades y clases
        probabilities = predictions['p1'].as_data_frame().values.flatten()
        predicted_classes = predictions['predict'].as_data_frame().values.flatten()
        
        # Agregar al DataFrame resultante
        X_transformed['predicted_prob'] = probabilities
        X_transformed['predicted_class'] = predicted_classes
        
        # Aplicar threshold si se especifica
        if self.best_threshold is not None:
            X_transformed['prediction'] = (
                X_transformed['predicted_prob'] >= self.best_threshold
            ).astype(int)
            logger.info(f"Umbral aplicado: {self.best_threshold}")
        
        logger.info(f"Predicciones completadas. Shape: {X_transformed.shape}")
        
        return X_transformed[['prediction']]
    
    def __del__(self):
        """Cleanup al destruir el objeto."""
        if self.h2o_initialized:
            h2o.cluster().shutdown()
            logger.info("H2O shutdown completado")


def create_prediction_pipeline(zone_mapper_df, sociodemographic_cols, 
                             columns_to_drop, power_params, 
                             h2o_model_path, best_threshold=None):
    """
    Crea un pipeline completo de scikit-learn para predicciones.
    
    Args:
        zone_mapper_df: DataFrame con mapeo sociodemográfico -> zona
        sociodemographic_cols: Columnas sociodemográficas
        columns_to_drop: Columnas a eliminar
        power_params: Parámetros para corrección de asimetría
        h2o_model_path: Ruta al modelo H2O
        best_threshold: Umbral para clasificación
        
    Returns:
        Pipeline de scikit-learn
    """
    pipeline = Pipeline([
        ('sociodemographic_mapper', SociodemographicToZoneTransformer(
            sociodemographic_cols=sociodemographic_cols,
            zone_mapper_df=zone_mapper_df
        )),
        ('column_dropper', ColumnDropper(
            columns_to_drop=columns_to_drop
        )),
        ('skewness_corrector', SkewnessCorrector(
            power_params=power_params
        )),
        ('h2o_predictor', H2OPredictor(
            model_path=h2o_model_path,
            best_threshold=best_threshold
        ))
    ])
    
    logger.info("Pipeline creado exitosamente")
    return pipeline




In [15]:
h2o_model_path = 'StackedEnsemble_BestOfFamily_1_AutoML_13_20251107_135752' #'h2o_automl_model'
best_threshold = 0.1
df_input = df[df['target']==1].head()

pipeline = create_prediction_pipeline(
    zone_mapper_df=preprocessor.zone_mapper_df,
    sociodemographic_cols=sociodemographic_cols,
    columns_to_drop=preprocessor.cols_to_drop_,
    power_params=preprocessor.power_params_,
    h2o_model_path=h2o_model_path,
    best_threshold=best_threshold
)

results = pipeline.fit_transform(df_input)

df_input['prediction'] = results.values

2025-11-08 03:14:13,915 - preprocessor_logger - INFO - [create_prediction_pipeline] - Pipeline creado exitosamente
2025-11-08 03:14:13,957 - preprocessor_logger - INFO - [__del__] - H2O shutdown completado
2025-11-08 03:14:13,958 - preprocessor_logger - INFO - [fit] - Fitteando SociodemographicToZoneTransformer
2025-11-08 03:14:13,959 - preprocessor_logger - INFO - [fit] - Zonas disponibles: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 14

2025-11-08 03:14:13,959 - preprocessor_logger - INFO - [transform] - Transformando columnas sociodemográficas a zona
2025-11-08 03:14:14,111 - preprocessor_logger - INFO - [transform] - Transformación completada. Shape: (5, 1777)
2025-11-08 03:14:14,111 - preprocessor_logger - INFO - [fit] - Fitteando ColumnDropper
2025-11-08 03:14:14,112 - preprocessor_logger - INFO - [fit] - Columnas a eliminar: ['PD_23', 'PD_34', 'PD_21', 'PD_28', 'PD_10', 'PD_20']
2025-11-08 03:14:14,112 - preprocessor_logger - INFO - [transform] - Eliminando columnas especificadas
2025-11-08 03:14:14,121 - preprocessor_logger - INFO - [transform] - Eliminadas 6 columnas
2025-11-08 03:14:14,121 - preprocessor_logger - INFO - [transform] - Transformación completada. Shape: (5, 1771)
2025-11-08 03:14:14,121 - preprocessor_logger - INFO - [fit] - Fitteando SkewnessCorrector
2025-11-08 03:14:14,122 - preprocessor_logger - INFO - [fit] - Columnas para corrección: ['PD_19', 'PD_9', 'PD_2', 'PD_7', 'PD_36', 'PD_33', 'PD_1

H2O session _sid_a54d closed.
Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,4 mins 10 secs
H2O_cluster_timezone:,America/Mexico_City
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.8
H2O_cluster_version_age:,30 days
H2O_cluster_name:,H2O_from_python_luis_caporal_ugmo7o
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.825 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


2025-11-08 03:14:14,155 - preprocessor_logger - INFO - [_initialize_h2o] - H2O inicializado
2025-11-08 03:14:14,267 - preprocessor_logger - INFO - [fit] - Modelo H2O cargado: StackedEnsemble_BestOfFamily_1_AutoML_1_20251108_31013
2025-11-08 03:14:14,268 - preprocessor_logger - INFO - [transform] - Realizando predicciones con H2O


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |

2025-11-08 03:14:14,808 - preprocessor_logger - INFO - [transform] - Umbral aplicado: 0.1
2025-11-08 03:14:14,808 - preprocessor_logger - INFO - [transform] - Predicciones completadas. Shape: (5, 1774)


███████████████████████████████████████████| (done) 100%


In [14]:
df_input

,SD_1,SD_2,SD_3,SD_4,SD_5,SD_6,SD_7,SD_8,SD_9,SD_10,SD_11,SD_12,SD_13,SD_14,SD_15,SD_16,SD_17,SD_18,SD_19,SD_20,SD_21,SD_22,SD_23,SD_24,SD_25,SD_26,SD_27,SD_28,SD_29,SD_30,SD_31,SD_32,SD_33,SD_34,SD_35,SD_36,SD_37,SD_38,SD_39,SD_40,SD_41,SD_42,SD_43,PD_0,PD_1,PD_2,PD_3,PD_4,PD_5,PD_6,PD_7,PD_8,PD_9,PD_10,PD_11,PD_12,PD_13,PD_14,PD_15,PD_16,PD_17,PD_18,PD_19,PD_20,PD_21,PD_22,PD_23,PD_24,PD_25,PD_26,PD_27,PD_28,PD_29,PD_30,PD_31,PD_32,PD_33,PD_34,PD_35,PD_36,PD_37,PD_38,PD_39,PD_40,PD_41,target,prediction
40,11,1,3,3,3,2,7,0.0,0.0,9,0.0,0.0,2.0,3.0,4,0.0,5.0,4.0,5.0,0.0,0.0,4.0,0.0,1.0,1.0,2.0,4.0,2.0,2.0,1.0,8.0,7.0,2.0,0.0,7.0,2.0,2.0,4,3.0,0.0,1.0,4.0,6.0,2.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,1
44,38,1,3,3,9,0,5,1.0,3.0,7,1.0,2.0,3.0,2.0,5,1.0,3.0,6.0,2.0,1.0,1.0,2.0,3.0,3.0,1.0,1.0,1.0,5.0,2.0,5.0,4.0,5.0,2.0,2.0,7.0,2.0,4.0,4,1.0,1.0,1.0,3.0,4.0,2.0,0.0,0.0,6.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1
56,12,1,3,2,3,0,6,0.0,3.0,7,2.0,0.0,1.0,2.0,6,2.0,4.0,4.0,1.0,0.0,0.0,5.0,2.0,1.0,0.0,3.0,2.0,4.0,0.0,0.0,9.0,9.0,0.0,0.0,5.0,4.0,2.0,4,4.0,0.0,0.0,4.0,7.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
96,9,1,2,3,3,0,6,1.0,2.0,7,1.0,1.0,5.0,1.0,4,5.0,4.0,0.0,4.0,0.0,0.0,5.0,1.0,0.0,1.0,8.0,0.0,0.0,0.0,9.0,0.0,8.0,1.0,1.0,4.0,5.0,0.0,5,4.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
97,36,1,2,4,8,2,4,2.0,2.0,7,0.0,2.0,2.0,4.0,3,1.0,5.0,4.0,1.0,0.0,0.0,3.0,1.0,4.0,1.0,1.0,4.0,4.0,0.0,2.0,7.0,5.0,3.0,2.0,5.0,4.0,2.0,5,3.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
